In [11]:
import polars as pl
from datetime import datetime


In [12]:
data = {
    "date": [datetime(2024, 7, 1), datetime(2024, 7, 2), datetime(2024, 7, 3), datetime(2024, 7, 4), datetime(2024, 7, 5)],
    "revenue": [100, 150, 200, 250, 300],
    "expenses": [80, 120, 160, 200, 240]
}

df = pl.DataFrame(data)
print(df)


shape: (5, 3)
┌─────────────────────┬─────────┬──────────┐
│ date                ┆ revenue ┆ expenses │
│ ---                 ┆ ---     ┆ ---      │
│ datetime[μs]        ┆ i64     ┆ i64      │
╞═════════════════════╪═════════╪══════════╡
│ 2024-07-01 00:00:00 ┆ 100     ┆ 80       │
│ 2024-07-02 00:00:00 ┆ 150     ┆ 120      │
│ 2024-07-03 00:00:00 ┆ 200     ┆ 160      │
│ 2024-07-04 00:00:00 ┆ 250     ┆ 200      │
│ 2024-07-05 00:00:00 ┆ 300     ┆ 240      │
└─────────────────────┴─────────┴──────────┘


In [13]:
# Filter Data
filtered_df = df.filter(
    pl.col("date").is_between(datetime(2024, 7, 2), datetime(2024, 7, 4))
)
print(filtered_df)


shape: (3, 3)
┌─────────────────────┬─────────┬──────────┐
│ date                ┆ revenue ┆ expenses │
│ ---                 ┆ ---     ┆ ---      │
│ datetime[μs]        ┆ i64     ┆ i64      │
╞═════════════════════╪═════════╪══════════╡
│ 2024-07-02 00:00:00 ┆ 150     ┆ 120      │
│ 2024-07-03 00:00:00 ┆ 200     ┆ 160      │
│ 2024-07-04 00:00:00 ┆ 250     ┆ 200      │
└─────────────────────┴─────────┴──────────┘


In [14]:
df_pandas = filtered_df.to_pandas()


In [16]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)


In [17]:
def build_prompt(df):
    prompt = "Analyze the financial data:\n"
    for row in df.itertuples(index=True):
        prompt += f"Date: {row.date.strftime('%Y-%m-%d')}, Revenue: ${row.revenue}, Expenses: ${row.expenses}\n"
    prompt += "Predict the revenue and expenses for the next date."
    return prompt

In [18]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


In [19]:
prompt = build_prompt(df_pandas)
answer = llm(prompt)
print(answer)

 To predict future financial data based on past performance is a common task in finance called time series forecasting. In this case, we observe that both revenues and expenses are increasing each day consistently by $50 and $40 respectively. This pattern suggests an arithmetic progression for the given figures:

- Revenue increases by $50 per day ($150 to $200 from 7th July; $200 to $250 on the following days)
  
- Expenses increase by $40 per day (from $120 to $160 and then to $200 respectively).

Assuming this pattern continues without change into the next period, we can predict revenue and expenses for 7th July as follows:

Date: 2024-07-05 (next day)
Predicted Revenue = Last known revenue + Daily increase in revenue
                      = $250 + $50
                      = **$300**.

Similarly, for expenses:
Predicted Expenses = Last observed expense + daily increase 
                     = $200 + $40
                     = **$240**.
